In [1]:
#
# Thermostat - This is the Python code used to demonstrate
# the functionality of the thermostat that we have prototyped throughout
# the course. 
#
# This code works with the test circuit that was built for module 7.
#
# Functionality:
#
# The thermostat has three states: off, heat, cool
#
# The lights will represent the state that the thermostat is in.
#
# If the thermostat is set to off, the lights will both be off.
#
# If the thermostat is set to heat, the Red LED will be fading in 
# and out if the current temperature is blow the set temperature;
# otherwise, the Red LED will be on solid.
#
# If the thermostat is set to cool, the Blue LED will be fading in 
# and out if the current temperature is above the set temperature;
# otherwise, the Blue LED will be on solid.
#
# One button will cycle through the three states of the thermostat.
#
# One button will raise the setpoint by a degree.
#
# One button will lower the setpoint by a degree.
#
# The LCD display will display the date and time on one line and
# alternate the second line between the current temperature and 
# the state of the thermostat along with its set temperature.
#
# The Thermostat will send a status update to the TemperatureServer
# over the serial port every 30 seconds in a comma delimited string
# including the state of the thermostat, the current temperature
# in degrees Fahrenheit, and the setpoint of the thermostat.
#
#------------------------------------------------------------------
# Change History
#------------------------------------------------------------------
# Version   |   Description
#------------------------------------------------------------------
#    1          Initial Development
#------------------------------------------------------------------

##
## Import necessary to provide timing in the main loop
##
from time import sleep
from datetime import datetime

##
## Imports required to allow us to build a fully functional state machine
##
# Note: My computer doesn't seen to have the statemachine module installed
#!pip install statemachine
from statemachine import StateMachine, State

##
## Imports necessary to provide connectivity to the 
## thermostat sensor and the I2C bus
##
import board
import adafruit_ahtx0

##
## These are the packages that we need to pull in so that we can work
## with the GPIO interface on the Raspberry Pi board and work with
## the 16x2 LCD display
##
# import board - already imported for I2C connectivity
import digitalio
import adafruit_character_lcd.character_lcd as characterlcd

## This imports the Python serial package to handle communications over the
## Raspberry Pi's serial port. 
import serial

##
## Imports required to handle our Button, and our PWMLED devices
##
from gpiozero import Button, PWMLED

##
## This package is necessary so that we can delegate the blinking
## lights to their own thread so that more work can be done at the
## same time
##
from threading import Thread

##
## This is needed to get coherent matching of temperatures.
##
from math import floor

##
## Imports needed for linear regression machine learning algorithm for Enhancement 3
##
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import csv
from sklearn import model_selection
from sklearn import linear_model

##
## DEBUG flag - boolean value to indicate whether or not to print 
## status messages on the console of the program
## 
DEBUG = True

##
## Create an I2C instance so that we can communicate with
## devices on the I2C bus.
##
i2c = board.I2C()

##
## Initialize our Temperature and Humidity sensor
##
thSensor = adafruit_ahtx0.AHTx0(i2c)

##
## Initialize our serial connection
##
## Because we imported the entire package instead of just importing Serial and
## some of the other flags from the serial package, we need to reference those
## objects with dot notation.
##
## e.g. ser = serial.Serial
##
ser = serial.Serial(
        port='/dev/ttyS0', # This would be /dev/ttyAM0 prior to Raspberry Pi 3
        baudrate = 115200, # This sets the speed of the serial interface in
                           # bits/second
        parity=serial.PARITY_NONE,      # Disable parity
        stopbits=serial.STOPBITS_ONE,   # Serial protocol will use one stop bit
        bytesize=serial.EIGHTBITS,      # We are using 8-bit bytes 
        timeout=1          # Configure a 1-second timeout
)

##
## Our two LEDs, utilizing GPIO 18, and GPIO 23
##
redLight = PWMLED(18)
blueLight = PWMLED(23)


##
## ManagedDisplay - Class intended to manage the 16x2 
## Display
##
## This code is largely taken from the work done in module 4, and
## converted into a class so that we can more easily consume the 
## operational capabilities.
##
class ManagedDisplay():
    ##
    ## Class Initialization method to setup the display
    ##
    def __init__(self):
        ##
        ## Setup the six GPIO lines to communicate with the display.
        ## This leverages the digitalio class to handle digital 
        ## outputs on the GPIO lines. There is also an analagous
        ## class for analog IO.
        ##
        ## You need to make sure that the port mappings match the
        ## physical wiring of the display interface to the 
        ## GPIO interface.
        ##
        ## compatible with all versions of RPI as of Jan. 2019
        ##
        self.lcd_rs = digitalio.DigitalInOut(board.D17)
        self.lcd_en = digitalio.DigitalInOut(board.D27)
        self.lcd_d4 = digitalio.DigitalInOut(board.D5)
        self.lcd_d5 = digitalio.DigitalInOut(board.D6)
        self.lcd_d6 = digitalio.DigitalInOut(board.D13)
        self.lcd_d7 = digitalio.DigitalInOut(board.D26)

        # Modify this if you have a different sized character LCD
        self.lcd_columns = 16
        self.lcd_rows = 2 

        # Initialise the lcd class
        self.lcd = characterlcd.Character_LCD_Mono(self.lcd_rs, self.lcd_en, 
                    self.lcd_d4, self.lcd_d5, self.lcd_d6, self.lcd_d7, 
                    self.lcd_columns, self.lcd_rows)

        # wipe LCD screen before we start
        self.lcd.clear()

    ##
    ## cleanupDisplay - Method used to cleanup the digitalIO lines that
    ## are used to run the display.
    ##
    def cleanupDisplay(self):
        # Clear the LCD first - otherwise we won't be abe to update it.
        self.lcd.clear()
        self.lcd_rs.deinit()
        self.lcd_en.deinit()
        self.lcd_d4.deinit()
        self.lcd_d5.deinit()
        self.lcd_d6.deinit()
        self.lcd_d7.deinit()
        
    ##
    ## clear - Convenience method used to clear the display
    ##
    def clear(self):
        self.lcd.clear()

    ##
    ## updateScreen - Convenience method used to update the message.
    ##
    def updateScreen(self, message):
        self.lcd.clear()
        self.lcd.message = message

    ##
    ## setupDisplay - Transfers the functionality for operating the lcd display
    ##  from the manageMyDisplay function to the ManagedDisplay class
    ##
    def setupDisplay(current_time, currentTempF, currentState, setPointTemp, counter):
        ## Setup display line 1
        lcd_line_1 = current_time.strftime("%b %d  %H:%M:%S\n")

        ## Setup Display Line 2
        if(counter < 6):
            lcd_line_2 = f"T:{currentTempF:0.1f}F"
        else:
            lcd_line_2 = f"ST:{currentState} Set:{setPointTemp}F"

        ## Output both lines together
        lcd_display = lcd_line_1 + lcd_line_2
        return lcd_display

    ## End class ManagedDisplay definition  

##
## Initialize our display
##
screen = ManagedDisplay()

##
## TemperatureMachine - This is our StateMachine implementation class.
## The purpose of this state machine is to manage the three states
## handled by our thermostat:
##
##  off
##  heat
##  cool
##
##
class TemperatureMachine(StateMachine):
    "A state machine designed to manage our thermostat"

    ##
    ## Define the three states for our machine.
    ##
    ##  off - nothing lit up
    ##  red - only red LED fading in and out
    ##  blue - only blue LED fading in and out
    ##
    off = State(initial = True)
    heat = State()
    cool = State()

    ##
    ## Default temperature setPoint is 72 degrees Fahrenheit
    ##
    setPoint = 72

    ##
    ## cycle - event that provides the state machine behavior
    ## of transitioning between the three states of our 
    ## thermostat
    ##
    cycle = (
        off.to(heat) |
        heat.to(cool) |
        cool.to(off)
    )

    ##
    ## on_enter_heat - Action performed when the state machine transitions
    ## into the 'heat' state
    ##
    def on_enter_heat(self):
        self.updateLights()
        if(DEBUG):
            print("* Changing state to heat")

    ##
    ## on_exit_heat - Action performed when the statemachine transitions
    ## out of the 'heat' state.
    ##
    def on_exit_heat(self):
        redLight.off()
    ##
    ## on_enter_cool - Action performed when the state machine transitions
    ## into the 'cool' state
    ##
    def on_enter_cool(self):
        self.updateLights()
        if(DEBUG):
            print("* Changing state to cool")
    
    ##
    ## on_exit_cool - Action performed when the statemachine transitions
    ## out of the 'cool' state.
    ##
    def on_exit_cool(self):
        blueLight.off()
    ##
    ## on_enter_off - Action performed when the state machine transitions
    ## into the 'off' state
    ##
    def on_enter_off(self):

        redLight.off()
        blueLight.off()
        if(DEBUG):
            print("* Changing state to off")
    
    ##
    ## processTempStateButton - Utility method used to send events to the 
    ## state machine. This is triggered by the button_pressed event
    ## handler for our first button
    ##
    def processTempStateButton(self):
        if(DEBUG):
            print("Cycling Temperature State")
        self.send("cycle")
    ##
    ## processTempIncButton - Utility method used to update the 
    ## setPoint for the temperature. This will increase the setPoint
    ## by a single degree. This is triggered by the button_pressed event
    ## handler for our second button
    ##
    def processTempIncButton(self):
        if(DEBUG):
            print("Increasing Set Point")

        self.setPoint = self.setPoint + 1
        self.updateLights()
    ##
    ## processTempDecButton - Utility method used to update the 
    ## setPoint for the temperature. This will decrease the setPoint
    ## by a single degree. This is triggered by the button_pressed event
    ## handler for our third button
    ##
    def processTempDecButton(self):
        if(DEBUG):
            print("Decreasing Set Point")

        self.setPoint = self.setPoint - 1
        self.updateLights()
    ##
    ## updateLights - Utility method to update the LED indicators on the 
    ## Thermostat
    ##
    def updateLights(self):
        ## Make sure we are comparing temperatires in the correct scale
        temp = floor(self.getFahrenheit())
        redLight.off()
        blueLight.off()
    
        ## Verify values for debug purposes
        if(DEBUG):
            print(f"State: {self.current_state.id}")
            print(f"SetPoint: {self.setPoint}")
            print(f"Temp: {temp}")

        # Determine visual identifiers
        if(self.current_state.id == 'heat'):
            ## Heat
            if(temp >= self.setPoint):
                ## Already at preferred temp
                redLight.on()
            else:
                ## Approaching preferred temp (heating)
                redLight.pulse()
        elif(self.current_state.id == 'cool'):
            ## Cool
            if(temp <= self.setPoint):
                ## Already at preferred temp
                blueLight.on()
            else:
                ## Approaching preferred temp (cooling)
                blueLight.pulse()
        else:
            ## Off
            pass
    ##
    ## run - kickoff the display management functionality of the thermostat
    ##
    def run(self):
        myThread = Thread(target=self.manageMyDisplay)
        myThread.start()

    ##
    ## Get the temperature in Fahrenheit
    ##
    def getFahrenheit(self):
        t = thSensor.temperature
        return (((9/5) * t) + 32)
    
    ##
    ##  Configure output string for the Thermostat Server
    ##
    def setupSerialOutput(self):
        output = f"{tsm.current_state.id},{self.getFahrenheit():0.1f},{self.setPoint}/n"
        return output
    
    ## Continue display output
    endDisplay = False

    ##
    ##  This function is designed to manage the LCD Display
    ##
    ## Note: Each function used in a program has a very small performance overhead.
    ##  This shouldn't be a serious issue, but this change is optimizing the
    ##  readability and maintainability of the program, not its efficiency. The program's
    ##  memory seems to overwrite the same few variables without storing any data,
    ##  except where it sends updates to a remote server which keeps track of data trends in its stead.
    ##  This program doesn't handle much data, and so might not be a good fit for the Algorithms and Data
    ##  Structures milestone. A better idea would be to develop a way to organize the data in the server,
    ##  if I had any way to access the server or change its operations.
    #
    ## Splitting the functionality of this function into lcd display operations and server operations seems
    ##  like it would be a good idea to improve the program's readability, but the two code segments are in
    ##  the same place for a reason -- they use the same clock to time their operations. Moving the display
    ##  functionality to another class would require running another while loop at the same time, which
    ##  could noticeably impact the program's efficiency. The program is designed as embedded software, which
    ##  means that making the code a bit less efficient in order to make it a bit more readable isn't a good
    ##  trade-off. This can be fixed by removing the loop functionality from the lcd display functionality and
    ##  simply calling the new setupDisplay function once every second.
    def manageMyDisplay(self):
        counter = 1
        altCounter = 1
        while not self.endDisplay:
            ## Only display if the DEBUG flag is set
            if(DEBUG):
                print("Processing Display Info...")
    
            ## Grab the current time(, temperature, state, and set point temperature)        
            current_time = datetime.now()
            current_temp = self.getFahrenheit()
            current_state = tsm.current_state.id
            setpoint_temp = tsm.setPoint

            ## Set display message
            lcd_display = screen.setupDisplay(current_time,
                                              current_temp,
                                              current_state,
                                              setpoint_temp,
                                              altCounter)
            altCounter = altCounter + 1
            if(altCounter >= 11):
                # Run the routine to update the lights every 10 seconds
                # to keep operations smooth
                self.updateLights()
                altCounter = 1
    
#           ## Setup display line 1
#            lcd_line_1 = current_time.strftime("%b %d  %H:%M:%S\n")
#
#           ## Setup Display Line 2
#            if(altCounter < 6):
#                lcd_line_2 = f"T:{self.getFahrenheit():0.1f}F"
#                altCounter = altCounter + 1
#            else:
#                lcd_line_2 = f"ST:{tsm.current_state.id} Set:{tsm.setPoint}F"
#                altCounter = altCounter + 1
#                if(altCounter >= 11):
#                    # Run the routine to update the lights every 10 seconds
#                    # to keep operations smooth
#                    self.updateLights()
#                    altCounter = 1
    
            ## Update Display
            screen.updateScreen(lcd_display)
    
            ## Update server every 30 seconds
            if(DEBUG):
               print(f"Counter: {counter}")
            if((counter % 30) == 0):
                ser.write(self.setupSerialOutput().encode())
                counter = 1
            else:
                counter = counter + 1
            sleep(1)

        ## Cleanup display
        screen.cleanupDisplay()

    ## End class TemperatureMachine definition

##
## Setup our State Machine
##
tsm = TemperatureMachine()
tsm.run()


##
## MachineLearning - Class intended to manage the linear regression
## chart for the accumulated temperature data and the machine learning
## algorithm that will be implemented for the CS-499 Enhancement Two:
## Algorithms and Data Structures
##
class MachineLearning():
    ##
    ## Build the lists
    ##
    stateList = []
    tempList = []
    setPointList = []
    
    ##
    ## Populate the lists with data entries from the serial connection
    ##
    def listsFromSerial(self):        
        try:
            while True:
                # Read a line from the serial port (adjust as per your data format)
                line = ser.readline().decode('utf-8').strip()
                
                if line: # Process only if the string is not empty
                    # Split the string by comma
                    state, temp, setpoint = line.split(',')
                    self.stateList.append(state)
                    self.tempList.append(float(temp))
                    self.setPointList.append(int(setpoint))

                    if(DEBUG):
                        print(f"Appended: {line}")
                else:
                    # If no data is received for 'timeout' seconds, break or handle as needed
                    print("No more data from serial.")
                    break
        except FileNotFoundError:
            print(f"Error: The serial connection was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")
        
    ##
    ## Linear regression model to implement the machine learning functionality
    ##
    def linearModel(self):
        ## Reshape the data to put each list entry into its own set of brackets,
        ## then split the data into training and testing data sets
        self.tempList = np.array(self.tempList).reshape(-1,1)
        self.setPointList = np.array(self.setPointList).reshape(-1,1)
        ## time variable in case of linear regression algorithm that uses time as a variable
        # time = []
        # for i in range(len(self.tempList)):
        #     time.append(i * 30)

        # tempTrain, tempTest, timeTrain, timeTest = model_selection.train_test_split(self.tempList, time, test_size = 0.2)
        tempTrain, tempTest, setPointTrain, setPointTest = model_selection.train_test_split(self.tempList, self.setPointList, test_size = 0.2)

        ## Model the linear regression algorithm for setPointTemperature / currentTemperature
        model = linear_model.LinearRegression()
        model.fit(setPointTrain, tempTrain)
        accuracy = model.score(setPointTest, tempTest)
        
        if(DEBUG):
            print(f"Coefficient: {model.coef_}")
            print(f"Intercept: {model.intercept_}")
            print(f"Accuracy: {round(accuracy * 100, 2)}")
            
    ## End class MachineLearning definition

newThread = Thread(target=self.MachineLearning())
newThread.start()

##
## Configure our green button to use GPIO 24 and to execute
## the method to cycle the thermostat when pressed.
##
greenButton = Button(24)
## Cycle thermostat state
greenButton.when_pressed = tsm.processTempStateButton
##
## Configure our Red button to use GPIO 25 and to execute
## the function to increase the setpoint by a degree.
##
redButton = Button(25)
## Increase the setpoint by one degree Fahrenheit.
redButton.when_pressed = tsm.processTempIncButton
##
## Configure our Blue button to use GPIO 12 and to execute
## the function to decrease the setpoint by a degree.
##
blueButton = Button(12)
## Decrease the setpoint by one degree Fahrenheit
blueButton.when_pressed = tsm.processTempDecButton
##
## Setup loop variable
##
repeat = True

##
## Repeat until the user creates a keyboard interrupt (CTRL-C)
##
while repeat:
    try:
        ## wait
        sleep(30)

    except KeyboardInterrupt:
        ## Catch the keyboard interrupt (CTRL-C) and exit cleanly
        ## we do not need to manually clean up the GPIO pins, the 
        ## gpiozero library handles that process.
        print("Cleaning up. Exiting...")

        ## Stop the loop
        repeat = False
        
        ## Close down the display
        tsm.endDisplay = True
        sleep(1)

IndentationError: expected an indented block (2867879518.py, line 565)